# Talk2PowerSystem

This notebook demonstrates a Natural Language Querying (NLQ) system using [Graphwise GraphDB](https://graphdb.ontotext.com/) and [LangGraph Agents](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/).

## Read the configuration

In [ ]:
import yaml

with open("config.yaml", "r", encoding="utf-8") as f:
    config = yaml.safe_load(f.read())

## Setup logging

In [ ]:
import logging
import sys

logger = logging.getLogger('')
logger.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(logging.Formatter("%(asctime)s - %(levelname)s - %(message)s"))

logger.handlers.clear()
logger.addHandler(handler)

logging.getLogger("openai").setLevel(logging.ERROR)
logging.getLogger("httpx").setLevel(logging.ERROR)
logging.getLogger("httpcore").setLevel(logging.ERROR)

## Initialize a GraphDB client

In [ ]:
from ttyg.graphdb import GraphDB

graph = GraphDB(
    base_url=config["graphdb"]["base_url"],
    repository_id=config["graphdb"]["repository_id"],
)

## Initialize the LLM using Azure OpenAI

In [ ]:
from langchain_openai import AzureChatOpenAI

from ttyg.utils import set_env

set_env("AZURE_OPENAI_API_KEY")
model = AzureChatOpenAI(
    azure_endpoint=config["llm"]["azure_endpoint"],
    api_version=config["llm"]["api_version"],
    model_name=config["llm"]["model_name"],
    temperature=config["llm"]["temperature"],
    seed=config["llm"]["seed"],
    timeout=config["llm"]["timeout"],
)

## Define the tools

In [ ]:
from collections import OrderedDict
from pathlib import Path

from ttyg.tools import (
    AutocompleteSearchTool,
    NowTool,
    OntologySchemaAndVocabularyTool,
    SparqlQueryTool,
)

sparql_query_tool = SparqlQueryTool(
    graph=graph,
)

ontology_schema_query = Path(config["ontology"]["ontology_schema_query_path"]).read_text()
ontology_schema_and_vocabulary_tool = OntologySchemaAndVocabularyTool(
    graph=graph,
    ontology_schema_query=ontology_schema_query,
)

string_enumerations_query = Path(config["ontology"]["string_enumerations_query_path"]).read_text()
results = graph.eval_sparql_query(string_enumerations_query)
known_prefixes = graph.get_known_prefixes()
sorted_known_prefixes = OrderedDict(sorted(known_prefixes.items(), key=lambda x: len(x[1]), reverse=True))
string_enumerations_prompt = ""
for r in results["results"]["bindings"]:
    shorten_property = r["property"]["value"]
    for prefix, namespace in sorted_known_prefixes.items():
        if shorten_property.startswith(namespace):
            shorten_property = shorten_property.replace(namespace, prefix + ":")
            break
    string_enumerations_prompt += f"""The unique string values of the property {shorten_property} separated with `;` are: {r["unique_objects"]["value"]}. \n"""

autocomplete_search_tool = AutocompleteSearchTool(
    graph=graph,
    limit=5,
    property_path="<http://iec.ch/TC57/2013/CIM-schema-cim16#IdentifiedObject.name>",
)

now_tool = NowTool()


## Create the ReAct agent

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

instructions = f"""{config['prompts']['assistant_instructions']}

The ontology schema to use in SPARQL queries is:

```turtle
{ontology_schema_and_vocabulary_tool.schema_graph.serialize(format='turtle')}
```

{string_enumerations_prompt}
"""

agent_executor = create_react_agent(
    model=model,
    tools=[
        autocomplete_search_tool,
        sparql_query_tool,
        now_tool,
    ],
    state_modifier=instructions,
    checkpointer=MemorySaver(),
    # debug=True,
)

## Talk to the power system

Note, that at the moment the conversations history is not persisted and is kept in the memory. Upon shut down of the notebook, it will be lost.

In [ ]:
import time


def print_stream(agent, inputs, config, last_message_id: str = None) -> str:
    sum_input_tokens, sum_output_tokens, sum_total_tokens = 0, 0, 0

    start = time.time()
    for s in agent.stream(inputs, config, stream_mode="values"):
        messages = s["messages"]
        for message in reversed(messages):
            if message.id == last_message_id:
                break

            message.pretty_print()
            if hasattr(message, "usage_metadata"):
                usage_metadata = message.usage_metadata
                input_tokens, output_tokens, total_tokens = usage_metadata["input_tokens"], usage_metadata["output_tokens"], usage_metadata["total_tokens"]
                sum_input_tokens += input_tokens
                sum_output_tokens += output_tokens
                sum_total_tokens += total_tokens
                logging.debug(
                    f"Usage: input tokens: {input_tokens}, "
                    f"output tokens: {output_tokens}, "
                    f"total tokens: {total_tokens}")

        last_message_id = messages[-1].id

    logging.debug(
        f"Total usage: input tokens: {sum_input_tokens}, "
        f"output tokens: {sum_output_tokens}, "
        f"total tokens: {sum_total_tokens}"
    )
    logging.debug(
        f"Elapsed time: {time.time() - start:.2f} seconds"
    )
    return last_message_id

### Send consecutive questions in the same thread (conversation)

In [ ]:
conf = {"configurable": {"thread_id": "thread-123"}}
messages = {"messages": [("user", "List all transformers within Substation OSLO.")]}
last_message_id = print_stream(agent_executor, messages, conf)

In [ ]:
messages = {"messages": [("user", "Give me their descriptions")]}
last_message_id = print_stream(agent_executor, messages, conf, last_message_id=last_message_id)

### Or iterate over set of questions, each within a separate thread (conversation)

In [ ]:
questions = [
    "List all transformers within Substation OSLO.",
    "Liste alle transformatorer innenfor Nettstasjon OSLO.",
    "List all substations within bidding zone NO2 SGR",
    "Liste alle nettstasjoner innenfor budsone NO2 SGR",
    "List all substations that are connected via an AC-line or a DC line to substation named ASKER",
    "List opp alle understasjoner som er koblet via en AC-linje eller en DC-linje til understasjon kalt ASKER",
    "List all AC-lines that traverse bidding zones NO5 SGR and NO2 SGR",
    "List opp alle AC-linjer som krysser budsonene NO5 SGR og NO2 SGR",
    "Give me 3 measurements in congestion zone NO-ELSP-1",
    "Gi meg målinger i overbelastningssone NO-ELSP-1",
    "show how many resources are there by class",
    "List five analogs of type active power",
    "Give me 8 synchronous machines of type generator",
    "List transformers that are normally in service",
    "Give me 5 switches that are normally closed",
    "List all synchronous machines that have \"M1\" or \"M2\" in the name, but not \"300\"",
    "Find the PSR f1769e0c",
    "List all substations north of Trondheim",
]

for i, question in enumerate(questions):
    conf = {"configurable": {"thread_id": f"thread-{i}"}}
    messages = {"messages": [("user", question)]}
    print_stream(agent_executor, messages, conf)